In [ ]:
# run requirements.txt for all dependencies


import pandas as pd
import numpy as np
import glob
import os
import dask
import dask.dataframe as dd
import dask.array as da
#import scripts.dask_impl as di
import re
# from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as sfm

import seaborn as sns
from matplotlib.widgets import Slider, Button, RadioButtons
from scipy import interp
from scipy.optimize import fsolve
from scipy.stats import chi2_contingency, ttest_ind
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from statsmodels.formula.api import ols


from IPython.display import display_html

from scipy import stats

%matplotlib inline
plt.style.use('ggplot')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# SET GLOBAL VARIABLES
RAW_PATH = '../data/raw'
# '/content/drive/Shared drives/DS4A_2021/Team74/data/raw'
# STAGE_FILE = '../data/stage/loan_files_stage.csv'
STAGE_FILE = '../data/stage/loan_w_all_demog.csv'
# STAGE_FILE = '../data/stage/fl.csv'
# '/content/drive/Shared drives/DS4A_2021/Team74/data/stage/loan_files_stage.csv'
NAICS_CODES = '../data/meta/naics-codes.csv'
PROFIT_STATUS = '../data/meta/profit_status.csv'
REPORT_NAME_DIRTY = 'PPP PROFILE BEFORE CLEANING'
REPORT_NAME_CLEAN = 'PPP PROFILE AFTER CLEANING'
REPORT_LOCATION = '../data/meta/'
REPORT_DIRTY_SAVE = 'PPP_PROFILE_BEFORE_CLEANING.html'
REPORT_CLEAN_SAVE = 'PPP_PROFILE_AFTER_CLEANING.html'

In [ ]:
# reloading 
df = pd.read_csv(STAGE_FILE)


##  PANDAS

In [ ]:
# merging the files PANDAS

all_files = glob.glob(RAW_PATH + "/*.csv") #returns list of all joined files 
dtypes = {'jobsreported': np.int64,'originatinglenderlocationid': np.int64,'sbaofficecode': np.int64,
          'servicinglenderlocationid': np.int64, 'FranchiseName':'category', 'NonProfit':'category'} #Dtype dict
li = []   

for filename in all_files:
    df_file = pd.read_csv(filename,dtype=dtypes, index_col=None, header=0)
    li.append(df_file)

df = pd.concat(li, axis=0, ignore_index=True)

## DATAFRAME INFO - run before and after cleaning

In [ ]:
# be careful with this, remove defaults so you can see all columns and rows
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [ ]:
#generate profile report - before cleaning and save it
profile = ProfileReport(df, minimal=True, title=REPORT_NAME_DIRTY)
profile.to_notebook_iframe()
profile.to_file(REPORT_LOCATION + REPORT_DIRTY_SAVE)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
# how many empty values in each column?
df.isna().sum()

In [ ]:
df.info()

In [ ]:
df['PAYROLL_PROCEED'].count()

In [ ]:
# not getting all my values filled in nonProfit for llc, sup?
df.loc[df['BusinessType'] == 'Limited Liability Company(LLC)']

In [ ]:
df.sample(50)

### UNIQUE VALUES 

In [ ]:

# get unique business types PANDAS
df.drop_duplicates(subset=['BusinessType'])

In [ ]:
# get unique nonprofit types PANDAS
df['NonProfit'].unique().tolist()

In [ ]:
# get unique ethnicity types PANDAS
df['Ethnicity'].unique().tolist()

In [ ]:
# get unique ethnicity types PANDAS
df['Gender'].unique().tolist()

In [ ]:
# get unique ethnicity types PANDAS
df['Veteran'].unique().tolist()

In [ ]:
# get unique race types PANDAS
df['Race'].unique().tolist()

In [ ]:
# get unique status types PANDAS
df['Status'].unique().tolist()

## CLEAN UP DATA

In [ ]:
#Remove commas from the column 
df['BorrowerName'] = df['BorrowerName'].str.replace(",","")
#strip leading and trailing space
df['BorrowerName'] = df['BorrowerName'].str.strip()
#Applying uppercase to a column
df['BorrowerName'] = df['BorrowerName'].str.upper()
#Remove commas from all other column
df = df.replace(',','', regex=True)

In [ ]:
# remove extra spaces in businessTypes
df['BusinessType'] = df['BusinessType'].str.replace('\s{2,}', ' ', regex=True)

In [ ]:
#Drop rows containing empty borrowerName and CurrentApprovalAmount  PANDAS

df['CurrentApprovalAmount'].replace('', np.nan, inplace=True)
df['BorrowerName'].replace('', np.nan, inplace=True)
df.dropna(subset=['CurrentApprovalAmount','BorrowerName'], inplace=True)

In [ ]:
# only when eliminating empty demog data
# df['Ethnicity'].replace('nan', np.nan, inplace=True)

In [ ]:
# get rid of weird Nan strings in status
# df['Status'].replace('Nan', np.nan, inplace=True)

In [ ]:
#replace N/A and Unanswered with nan in each column  - there's a faster way to do this
df = df.replace('Unanswered',np.nan, regex=True)
df = df.replace('N/A',np.nan, regex=True)

In [ ]:
#Remove last 4 digits of zip
df['BorrowerZip'] = df['BorrowerZip'].astype(str).str[:5]
df['ProjectZip'] = df['ProjectZip'].astype(str).str[:5]
df['ServicingLenderZip'] = df['ServicingLenderZip'].astype(str).str[:5]

In [ ]:
#Convert data types - isn't this getting set during conversion?   PANDAS
df['DateApproved'] = pd.to_datetime(df['DateApproved'])
df['LoanStatusDate'] = pd.to_datetime(df['LoanStatusDate']) 
df['LoanStatus'] = df.LoanStatus.astype('category')
df['BorrowerState'] = df.BorrowerState.astype('category')
df['Race'] = df.Race.astype('category')
df['BorrowerZip'] = df.BorrowerZip.astype('category')
df['ServicingLenderZip'] = df.ServicingLenderZip.astype('category')
df['ProjectZip'] = df.ProjectZip.astype('category')

In [ ]:
# NaN BusinessType converted to ‘unknown’
df['BusinessType'] = df['BusinessType'].fillna('UNKNOWN')

In [ ]:
# drop dupe LoanNumber - PANDAS
df = df.drop_duplicates(subset=['LoanNumber'], keep='first')

In [ ]:
# make empty PAYROLL_PROCEED == 0
df['PAYROLL_PROCEED'] = df['PAYROLL_PROCEED'].fillna(0)

In [ ]:
# fill in the NonProfit column with dictionary of values we've mapped
nonprofit_status = pd.read_csv(PROFIT_STATUS, skipinitialspace=True)
mapping = dict(nonprofit_status[['BusinessType', 'nonprofit']].values)
df['NonProfit'] = df['BusinessType'].map(mapping)

# ADD NEW COLUMNS

In [ ]:
# create a new column - convert naics code to sector    ^([0-9]{2})
# lets do this the hard way, create a new column called sector, populate it with naics - first 2 digits

search = []  
regex_code = '^([0-9]{2})'
for values in df['NAICSCode'].fillna('').apply(str):
    try:
        search.append(re.search(r'^([0-9]{2})', values).group())
    except AttributeError:
        search.append(re.search(r'^([0-9]{2})', values))
df['sector'] = search
df['sector'].fillna(0, inplace=True)
df['sector'] = df.sector.astype('int64')
# df['sector'].unique()

In [ ]:
# now lets convert those to the sector values from our naic file - good lord this was work
naic = pd.read_csv(NAICS_CODES ,skipinitialspace=True)
naic.info()
mapping = dict(naic[['Sector','Definition']].values)
df['sector'] = df['sector'].map(mapping)

In [ ]:
# add new column - payroll as percent of loan, 1 dec places
df['payrollPctLoan'] = ((df['PAYROLL_PROCEED']/df['CurrentApprovalAmount']).round(decimals=1) * 100).astype(float)

In [ ]:
#generate profile report - after cleaning and save it
profile = ProfileReport(df, minimal=True, title=REPORT_NAME_CLEAN)
profile.to_notebook_iframe()
profile.to_file(REPORT_LOCATION + REPORT_CLEAN_SAVE)

In [ ]:
# write merged dataframe to drive
# df.to_csv (STAGE_FILE, index = None, header=True) 
# df.to_csv ('../data/stage/loan_w_all_demog.csv', index = None, header=True) 
# df.to_csv ('../data/stage/fl_w_all_demog.csv', index = None, header=True) 

In [ ]:
df.isna().sum()

In [ ]:
df.count

In [ ]:
df.dropna(subset=['Race'], inplace=True)

In [ ]:
df.dropna(subset=['Gender'], inplace=True)

In [ ]:
df.dropna(subset=['Veteran'], inplace=True)

In [ ]:
df.dropna(subset=['Ethnicity'], inplace=True)

In [ ]:
df.dropna(subset=['Status'], inplace=True)

In [ ]:
df_correlations = df.corr()
# mask the upper half for viz purposes
mask = np.zeros_like(df_correlations, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# draw heatmap with mask and correct aspect ratio
plt.figure(figsize = (10,10))
plt.title('PPP Univariate Correlation Heatmap')
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(df_correlations, mask=mask, vmax=1, vmin=-1, cmap=cmap, center=0, square=True, 
            linewidths=.5, cbar_kws={'shrink': .5});

In [ ]:
# demog vs loan amount
def demog_vs_num(demog, num_value):
    df[[demog,num_value]].groupby(demog).mean().sort_values('CurrentApprovalAmount', 
                                                            ascending=False).plot.bar(figsize=(20,10),rot=0)
    plt.title(demog + " vs Mean Approval Amount")
    plt.xticks(fontsize=25)
    plt.yticks(fontsize=25)
    plt.xlabel(demog, fontsize=25)
    plt.ylabel('Current Approval Amount', fontsize=25)
    plt.title(demog + " vs Mean Approval Amount", fontsize=50)
    

In [ ]:
demog_vs_num('Race', 'CurrentApprovalAmount')

In [ ]:
demog_vs_num('Gender', 'CurrentApprovalAmount')

In [ ]:
demog_vs_num('Veteran', 'CurrentApprovalAmount')

In [ ]:
demog_vs_num('Ethnicity', 'CurrentApprovalAmount')

In [ ]:
# group by multiple demogs
def multiple_demogs(first, second):
        
    ax = df[[first, second,'CurrentApprovalAmount']].groupby([first, second])\
    .mean().sort_values('CurrentApprovalAmount', ascending=False).plot.bar(figsize=(75,20))
    
    # get actual numbers
    for p in ax.patches:                 
        ax.annotate(np.round(p.get_height(),decimals=1),
        (p.get_x()+p.get_width()/2., p.get_height()), 
        ha='center', va='center',xytext=(0, 25), 
        textcoords='offset points', fontsize=30)
    
    plt.title(first + '/' + second + " vs Mean Approval Amount", fontsize=75)
    plt.xticks(rotation=45,ha='right',fontsize=50)
    plt.yticks(fontsize=50)
    plt.xlabel(first + '/' + second,fontsize=50)
    plt.ylabel('Current Approval Amount', fontsize=50)
    plt.show()

In [ ]:
# race and gender
multiple_demogs('Race', 'Gender')

In [ ]:
# get the means as df
race_gender_app_mean = df[['Race', 'Gender','CurrentApprovalAmount']].groupby(['Race', 'Gender'])\
    .mean().sort_values('CurrentApprovalAmount', ascending=False).round(1)


In [ ]:
race_gender_app_mean.to_csv ('../data/stage/race_gender_app_mea.csv', header=True) 

In [ ]:
multiple_demogs('Ethnicity', 'Gender')

In [ ]:
# convert status to number so we can get liklihoods
